In [45]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import greycomatrix, greycoprops
from skimage.io import imread, imshow
from skimage.color import rgb2gray
from sklearn.preprocessing import LabelEncoder
import cv2
import os

## 1. Feature Extraction 

Haralick Features

In [58]:
def extract_haralick_features(grayscale_image, args):
    distances = args['distances']
    angles = args['angles']
    glcm = greycomatrix(grayscale_image, distances, angles)
    features = [
        np.mean(greycoprops(glcm, 'contrast')),
        np.mean(greycoprops(glcm, 'dissimilarity')),
        np.mean(greycoprops(glcm, 'homogeneity')),
        np.mean(greycoprops(glcm, 'energy')),
        np.mean(greycoprops(glcm, 'correlation')),
        np.mean(greycoprops(glcm, 'ASM')),
    ]
    return features

def label_encode(y):
    le = LabelEncoder()
    le.fit(y)
    return (le.transform(y), le.classes_)

def get_dir_contents(path):
    cts = os.listdir(path)
    return [c for c in cts if c[0] != '.']

def get_X_y_vectors(path, feature_extraction_function, args):
    X = []
    y = []
    for label in get_dir_contents(path):
        for file_name in get_dir_contents(path + label + "/"):
            image = cv2.imread(path + label + "/" + file_name, cv2.IMREAD_GRAYSCALE)
            features = feature_extraction_function(image, args)
            X.append(features)
            y.append(label)
    y, classes_ = label_encode(y)
    return (np.array(X), y, classes_)

In [132]:
TRAIN_IMAGES_PATH = 'dataset/train/'
TEST_IMAGES_PATH = 'dataset/test/'

haralick_args = {
    'distances': [1], 
    'angles': [0, np.pi/4, np.pi/2, 3*np.pi/4]
}

X_train, y_train, train_classes_ = get_X_y_vectors(
    TRAIN_IMAGES_PATH, extract_haralick_features, haralick_args
)
X_test, y_test, test_classes_ = get_X_y_vectors(
    TEST_IMAGES_PATH, extract_haralick_features, haralick_args
)


print(f"X_train: {X_train.shape}\ty_train: {y_train.shape}")
print(f"X_test: {X_test.shape}\ty_test: {y_test.shape}")
print(f"Train classes: {train_classes_}\nTest classes: {test_classes_}")

X_train: (800, 6)	y_train: (800,)
X_test: (300, 6)	y_test: (300,)
Train classes: ['coast' 'dead_sargassum' 'sargassum' 'sky' 'water']
Test classes: ['coast' 'dead_sargassum' 'sargassum' 'sky' 'water']


## 2. Model Training and Evaluation

In [165]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix
import pandas as pd

def evaluate(model, X_test, y_true, y_pred, target_names, model_name):
    print(f"----------—-------------—-------------—---{model_name}----------—-------------—-------------—---")
    print(classification_report(y_true, y_pred, target_names=target_names))
    plot_confusion_matrix(
        model, X_test, y_true, display_labels=target_names, cmap=plt.cm.Blues
    )
    plt.title(f"{model_name} confusion matrix")
    plt.show()

In [168]:
classifiers = [
    KNeighborsClassifier(n_neighbors=5, weights='distance'),
    SVC(),
    GaussianProcessClassifier(),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, max_features=5),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(random_state=0),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()
]
names = ["Nearest Neighbors", "SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Network", "AdaBoost",
         "Naive Bayes", "QDA"]

scores = []

for name, classifier in zip(names, classifiers):
    classifier.fit(X_train, y_train)
    score = classifier.score(X_test, y_test)
    y_pred = classifier.predict(X_test)
    scores.append(score)

df = pd.DataFrame({'classifier': names,'score': scores})
df = df.sort_values(by='score', ascending=False)
df

,classifier,score
4,Random Forest,0.683333
3,Decision Tree,0.640000
6,AdaBoost,0.616667
8,QDA,0.616667
5,Neural Network,0.610000
2,Gaussian Process,0.603333
0,Nearest Neighbors,0.573333
1,SVM,0.543333
7,Naive Bayes,0.493333


In [176]:
best_classifiers_df = df.iloc[:2]
for idx in np.array(best_classifiers_df.index):
    name = names[idx]
    clf = classifiers[idx]
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(f"-------------------------{name}-------------------------")
    print(classification_report(y_test, y_pred, target_names=train_classes_))

-------------------------Random Forest-------------------------
                precision    recall  f1-score   support

         coast       0.80      0.95      0.87       116
dead_sargassum       0.62      0.36      0.45        45
     sargassum       0.52      0.25      0.33        53
           sky       0.85      0.79      0.82        29
         water       0.54      0.81      0.65        57

      accuracy                           0.69       300
     macro avg       0.67      0.63      0.62       300
  weighted avg       0.68      0.69      0.67       300

-------------------------Decision Tree-------------------------
                precision    recall  f1-score   support

         coast       0.76      0.96      0.85       116
dead_sargassum       0.47      0.18      0.26        45
     sargassum       0.28      0.13      0.18        53
           sky       0.84      0.72      0.78        29
         water       0.51      0.77      0.61        57

      accuracy             

In [179]:
best_classifiers_df = df.iloc[2:4]
for idx in np.array(best_classifiers_df.index):
    name = names[idx]
    clf = classifiers[idx]
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(f"-------------------------{name}-------------------------")
    print(classification_report(y_test, y_pred, target_names=train_classes_))

-------------------------AdaBoost-------------------------
                precision    recall  f1-score   support

         coast       0.81      0.84      0.83       116
dead_sargassum       0.34      0.42      0.38        45
     sargassum       0.25      0.04      0.07        53
           sky       0.90      0.62      0.73        29
         water       0.51      0.84      0.63        57

      accuracy                           0.62       300
     macro avg       0.56      0.55      0.53       300
  weighted avg       0.59      0.62      0.58       300

-------------------------QDA-------------------------
                precision    recall  f1-score   support

         coast       0.99      0.59      0.74       116
dead_sargassum       0.44      0.89      0.59        45
     sargassum       0.38      0.11      0.17        53
           sky       0.75      0.72      0.74        29
         water       0.51      0.86      0.64        57

      accuracy                           0

In [181]:
best_classifiers_df = df.iloc[4:6]
for idx in np.array(best_classifiers_df.index):
    name = names[idx]
    clf = classifiers[idx]
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(f"-------------------------{name}-------------------------")
    print(classification_report(y_test, y_pred, target_names=train_classes_))

-------------------------Neural Network-------------------------
                precision    recall  f1-score   support

         coast       0.71      0.98      0.82       116
dead_sargassum       0.00      0.00      0.00        45
     sargassum       1.00      0.02      0.04        53
           sky       0.48      0.72      0.58        29
         water       0.49      0.81      0.61        57

      accuracy                           0.61       300
     macro avg       0.53      0.51      0.41       300
  weighted avg       0.59      0.61      0.50       300



/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


-------------------------Gaussian Process-------------------------
                precision    recall  f1-score   support

         coast       0.76      0.77      0.76       116
dead_sargassum       0.41      0.42      0.42        45
     sargassum       0.50      0.19      0.27        53
           sky       0.59      0.66      0.62        29
         water       0.52      0.77      0.62        57

      accuracy                           0.60       300
     macro avg       0.56      0.56      0.54       300
  weighted avg       0.60      0.60      0.58       300



In [182]:
best_classifiers_df = df.iloc[6:8]
for idx in np.array(best_classifiers_df.index):
    name = names[idx]
    clf = classifiers[idx]
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(f"-------------------------{name}-------------------------")
    print(classification_report(y_test, y_pred, target_names=train_classes_))

-------------------------Nearest Neighbors-------------------------
                precision    recall  f1-score   support

         coast       0.77      0.78      0.77       116
dead_sargassum       0.38      0.42      0.40        45
     sargassum       0.33      0.17      0.22        53
           sky       0.65      0.76      0.70        29
         water       0.44      0.56      0.50        57

      accuracy                           0.57       300
     macro avg       0.51      0.54      0.52       300
  weighted avg       0.56      0.57      0.56       300

-------------------------SVM-------------------------
                precision    recall  f1-score   support

         coast       0.71      0.94      0.81       116
dead_sargassum       0.00      0.00      0.00        45
     sargassum       0.00      0.00      0.00        53
           sky       0.00      0.00      0.00        29
         water       0.37      0.95      0.53        57

      accuracy                   

/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [186]:
best_classifiers_df = df.iloc[8:9]
for idx in np.array(best_classifiers_df.index):
    name = names[idx]
    clf = classifiers[idx]
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(f"-------------------------{name}-------------------------")
    print(classification_report(y_test, y_pred, target_names=train_classes_))

-------------------------Naive Bayes-------------------------
                precision    recall  f1-score   support

         coast       0.96      0.39      0.55       116
dead_sargassum       0.34      0.80      0.48        45
     sargassum       0.20      0.02      0.03        53
           sky       0.42      0.83      0.56        29
         water       0.49      0.74      0.59        57

      accuracy                           0.49       300
     macro avg       0.48      0.55      0.44       300
  weighted avg       0.59      0.49      0.46       300

